In [ ]:
import dualcodec
import torchaudio
from IPython.display import Audio

In [ ]:
model_id = "12hz_v1"

In [ ]:
path_1 = "output_checkpoints_3/dualcodec_experiments_causal_decoder/checkpoint/epoch-0000_step-0052000_loss-210.759888-dualcodec_experiments_causal_decoder"

In [ ]:
# use strict=False to load the model when making changes to the model layers (which changes the names in the state dictionairy)
# This is only to load random weights and confirm that the forward pass works properly

dualcodec_model = dualcodec.get_model(model_id, pretrained_model_path=path_1, is_checkpoint=True, strict=False) 

In [ ]:
# If you switch from InferenceWhisper to Inference (normal one with w2v) you have to change the model semantic_repr_dim to 1024 in 
# conf/model/dualcodec_12hz_16384_4096_8vq.yaml

dualcodec_inference = dualcodec.InferenceWhisper(dualcodec_model=dualcodec_model)

In [ ]:
dualcodec_inference.semantic_cfg.semantic_model.encoder.causal_mask[0][0][1][:10]

In [ ]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [ ]:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio, n_quantizers=8)

In [ ]:
print(semantic_codes.shape)
print(acoustic_codes.shape)

In [ ]:
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)